In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader 

from tqdm import tqdm
import heapq
import csv

import numpy as np
import random
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
import pandas as pd
import wandb

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [2]:
def data_load(path):
    # input - English
    # output - Urdu
    df = pd.read_csv(path,sep='\t', header=None)
    input_data = df[1].tolist()
    output_data = df[0].tolist()
    return input_data, output_data
    
def create_char_set(train, val):
    char_set = set()
    for word in train:
        for char in str(word):
            char_set.add(char)
    for word in val:
        for char in str(word):
            char_set.add(char)
    return char_set

def check_for_floats(data_list):
    float_values = []
    for item in data_list:
        if isinstance(item, float):
            float_values.append(item)
    return float_values

In [3]:
train_input1, train_output = data_load("/kaggle/input/dakshina-dataset/dakshina_dataset_v1.0/ur/lexicons/ur.translit.sampled.train.tsv")
val_input, val_output = data_load("/kaggle/input/dakshina-dataset/dakshina_dataset_v1.0/ur/lexicons/ur.translit.sampled.dev.tsv")
test_input, test_output = data_load("/kaggle/input/dakshina-dataset/dakshina_dataset_v1.0/ur/lexicons/ur.translit.sampled.test.tsv") 

print("Number of training samples: ", len(train_input1))
print("Number of validation samples: ", len(val_input))
print("Number of test samples: ", len(test_input))


Number of training samples:  106260
Number of validation samples:  10424
Number of test samples:  10517


In [4]:
float_values=check_for_floats(train_input1)
print(float_values)

[nan, nan]


In [5]:
train_input = [x for x in train_input1 if not isinstance(x,float)]
print("Number of training samples: ", len(train_input))

Number of training samples:  106258


In [6]:
max_seq_eng = len(max(train_input+val_input+test_input, key=len))
max_seq_ur = len(max(train_output+val_output+test_output, key=len))
print("Length of the longest English word in corpus:",max_seq_eng)
print("Length of the longest Urdu word in corpus::",max_seq_ur)


Length of the longest English word in corpus: 21
Length of the longest Urdu word in corpus:: 14


In [7]:
import torch
from torch.utils.data import DataLoader, Dataset
import pandas as pd

def load_data(input_data, output_data, batch_size=32):
    """
    Prepares character-level transliteration data for sequence-to-sequence modeling.

    Args:
        input_data (list or Series): Romanized source words (e.g., Urdu in Latin script).
        output_data (list or Series): Target words in Devanagari/Urdu script.
        batch_size (int): Batch size for the DataLoader.

    Returns:
        dataset (Dataset): Custom PyTorch dataset for transliteration.
        dataloader (DataLoader): DataLoader for iterating over the dataset.
        input_vocab (dict): Character-to-index mapping for input characters.
        target_vocab (dict): Character-to-index mapping for target characters.
        max_input_len (int): Maximum input sequence length.
        max_target_len (int): Maximum target sequence length.
    """

    # Determine the maximum lengths for padding
    max_input_len = len(max(input_data, key=len))
    max_target_len = len(max(output_data, key=len))

    # Initialize character vocabularies with special tokens
    input_vocab = {'<pad>': 0, '<sos>': 1, '<eos>': 2}
    target_vocab = {'<pad>': 0, '<sos>': 1, '<eos>': 2}
    next_index = 3

    # Extract all unique characters from the dataset
    all_input_chars = ''.join(input_data)
    all_target_chars = ''.join(output_data)

    # Create input vocabulary
    for char in sorted(set(all_input_chars)):
        input_vocab[char] = next_index
        next_index += 1

    # Reset index for building target vocabulary
    next_index = 3
    for char in sorted(set(all_target_chars)):
        if char not in target_vocab:
            target_vocab[char] = next_index
            next_index += 1

    # Tokenize input characters and pad to max_input_len
    def tokenize_input(word, vocab, max_len):
        token_ids = [vocab[char] for char in word if char in vocab]
        padded = token_ids[:max_len] + [vocab['<pad>']] * (max_len - len(token_ids))
        return torch.tensor(padded)

    # Tokenize target characters with <sos> and <eos>, then pad
    def tokenize_target(word, vocab, max_len):
        token_ids = [vocab[char] for char in word if char in vocab]
        padded = [vocab['<sos>']] + token_ids[:max_len] + [vocab['<eos>']]
        padded += [vocab['<pad>']] * (max_len + 2 - len(padded))  # +2 for <sos> and <eos>
        return torch.tensor(padded)

    # Define a custom dataset for transliteration pairs
    class TransliterationDataset(Dataset):
        def __init__(self, input_words, target_words, input_vocab, target_vocab, max_input_len, max_target_len):
            self.input_words = input_words
            self.target_words = target_words
            self.input_vocab = input_vocab
            self.target_vocab = target_vocab
            self.max_input_len = max_input_len
            self.max_target_len = max_target_len

        def __len__(self):
            return len(self.input_words)

        def __getitem__(self, idx):
            input_word = self.input_words[idx]
            target_word = self.target_words[idx]

            # Convert to tensor of indices
            input_tensor = tokenize_input(input_word, self.input_vocab, self.max_input_len)
            target_tensor = tokenize_target(target_word, self.target_vocab, self.max_target_len)

            return input_tensor, target_tensor

    # Create dataset and dataloader
    dataset = TransliterationDataset(input_data, output_data,
                                     input_vocab, target_vocab,
                                     max_input_len, max_target_len)

    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    return dataset, dataloader, input_vocab, target_vocab, max_input_len, max_target_len


In [8]:
dataset, dataloader, input_vocab, target_vocab, max_input_len, max_target_len = load_data(train_input+val_input+test_input, train_output+val_output+test_output,batch_size = 64)
print('All English Characters:\n',input_vocab,'\n All Urdu Characters:\n', target_vocab,'\n Length of  longest English word:', max_input_len,'\n Length of  longest Urdu word:', max_target_len) 

All English Characters:
 {'<pad>': 0, '<sos>': 1, '<eos>': 2, 'a': 3, 'b': 4, 'c': 5, 'd': 6, 'e': 7, 'f': 8, 'g': 9, 'h': 10, 'i': 11, 'j': 12, 'k': 13, 'l': 14, 'm': 15, 'n': 16, 'o': 17, 'p': 18, 'q': 19, 'r': 20, 's': 21, 't': 22, 'u': 23, 'v': 24, 'w': 25, 'x': 26, 'y': 27, 'z': 28} 
 All Urdu Characters:
 {'<pad>': 0, '<sos>': 1, '<eos>': 2, 'ء': 3, 'آ': 4, 'ؤ': 5, 'ئ': 6, 'ا': 7, 'ب': 8, 'ت': 9, 'ث': 10, 'ج': 11, 'ح': 12, 'خ': 13, 'د': 14, 'ذ': 15, 'ر': 16, 'ز': 17, 'س': 18, 'ش': 19, 'ص': 20, 'ض': 21, 'ط': 22, 'ظ': 23, 'ع': 24, 'غ': 25, 'ف': 26, 'ق': 27, 'ك': 28, 'ل': 29, 'م': 30, 'ن': 31, 'ه': 32, 'و': 33, 'ي': 34, 'ً': 35, 'َ': 36, 'ُ': 37, 'ِ': 38, 'ّ': 39, 'ٗ': 40, 'ٰ': 41, 'ٹ': 42, 'پ': 43, 'چ': 44, 'ڈ': 45, 'ڑ': 46, 'ژ': 47, 'ک': 48, 'گ': 49, 'ں': 50, 'ھ': 51, 'ہ': 52, 'ۃ': 53, 'ی': 54, 'ے': 55, 'ۓ': 56} 
 Length of  longest English word: 21 
 Length of  longest Urdu word: 14


In [9]:
# Encoder module for sequence-to-sequence model with attention
class Encoder(nn.Module):
    def __init__(self, input_vocab_size, hidden_size, embedding_size, num_layers=1, dropout=0.5, cell_type='gru', bidirectional=False):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.cell_type = cell_type
        self.bidirectional = bidirectional

        # Embedding layer to convert input tokens into dense vectors
        self.embedding = nn.Embedding(input_vocab_size, embedding_size)
        self.dropout = nn.Dropout(dropout)

        # RNN layer choice based on specified cell type
        if cell_type == 'lstm':
            self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, dropout=dropout, bidirectional=bidirectional, batch_first=True)
        elif cell_type == 'gru':
            self.rnn = nn.GRU(embedding_size, hidden_size, num_layers, dropout=dropout, bidirectional=bidirectional, batch_first=True)
        else:
            self.rnn = nn.RNN(embedding_size, hidden_size, num_layers, dropout=dropout, bidirectional=bidirectional, batch_first=True)

    def forward(self, input_seq):
        # input_seq shape: (batch_size, seq_len)
        embedded = self.dropout(self.embedding(input_seq))  # (batch_size, seq_len, embedding_size)
        outputs, hidden = self.rnn(embedded)  # outputs: all hidden states; hidden: final state(s)

        if self.cell_type == 'lstm':
            h_n, c_n = hidden  # hidden state and cell state
            if self.bidirectional:
                # Concatenate final forward and backward states
                hidden = (
                    torch.cat([h_n[-2], h_n[-1]], dim=1).unsqueeze(0), 
                    torch.cat([c_n[-2], c_n[-1]], dim=1).unsqueeze(0)
                )
            else:
                hidden = (h_n[-1].unsqueeze(0), c_n[-1].unsqueeze(0))
        else:
            if self.bidirectional:
                hidden = torch.cat([hidden[-2], hidden[-1]], dim=1).unsqueeze(0)
            else:
                hidden = hidden[-1].unsqueeze(0)

        return outputs, hidden  # outputs for attention; hidden for decoder init


# Attention mechanism to compute context vector
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.attn_layer = nn.Linear(hidden_size * 2, hidden_size)
        self.v = nn.Parameter(torch.rand(hidden_size))  # vector for computing attention scores

    def forward(self, decoder_hidden, encoder_outputs):
        # decoder_hidden shape: (batch_size, hidden_size)
        # encoder_outputs shape: (batch_size, seq_len, hidden_size)
        seq_len = encoder_outputs.shape[1]

        # Repeat decoder hidden state across all time steps of encoder output
        repeated_hidden = decoder_hidden.unsqueeze(1).repeat(1, seq_len, 1)

        # Concatenate and compute attention energy
        energy = torch.tanh(self.attn_layer(torch.cat((repeated_hidden, encoder_outputs), dim=2)))  # (batch, seq_len, hidden_size)
        v = self.v.repeat(encoder_outputs.size(0), 1).unsqueeze(1)  # (batch, 1, hidden_size)

        # Compute attention scores and normalize with softmax
        attn_scores = torch.bmm(v, energy.transpose(1, 2)).squeeze(1)  # (batch, seq_len)
        return torch.softmax(attn_scores, dim=1)  # normalized weights


# Decoder with attention mechanism
class DecoderWithAttention(nn.Module):
    def __init__(self, hidden_size, embedding_size, output_vocab_size, attention, num_layers=1, dropout=0.5, cell_type='gru'):
        super(DecoderWithAttention, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.cell_type = cell_type
        self.attention = attention

        self.embedding = nn.Embedding(output_vocab_size, embedding_size)
        self.dropout = nn.Dropout(dropout)

        rnn_input_size = hidden_size + embedding_size
        if cell_type == 'lstm':
            self.rnn = nn.LSTM(rnn_input_size, hidden_size, num_layers, dropout=dropout, batch_first=True)
        elif cell_type == 'gru':
            self.rnn = nn.GRU(rnn_input_size, hidden_size, num_layers, dropout=dropout, batch_first=True)
        else:
            self.rnn = nn.RNN(rnn_input_size, hidden_size, num_layers, dropout=dropout, batch_first=True)

        # Fully connected layer to output logits over target vocabulary
        self.fc_out = nn.Linear(hidden_size * 2, output_vocab_size)

    def forward(self, input_token, hidden, encoder_outputs):
        # input_token: (batch_size)
        embedded = self.dropout(self.embedding(input_token)).unsqueeze(1)  # (batch_size, 1, embedding_size)

        if self.cell_type == 'lstm':
            hidden_state = hidden[0][-1]  # take last hidden state for attention
        else:
            hidden_state = hidden[-1]

        # Compute context vector from attention
        attn_weights = self.attention(hidden_state, encoder_outputs)  # (batch_size, seq_len)
        context = torch.bmm(attn_weights.unsqueeze(1), encoder_outputs)  # (batch, 1, hidden_size)

        # Combine context with embedded input
        rnn_input = torch.cat((embedded, context), dim=2)  # (batch_size, 1, hidden+embedding)

        # Pass through RNN
        if self.cell_type == 'lstm':
            output, (hidden, cell) = self.rnn(rnn_input, hidden)
        else:
            output, hidden = self.rnn(rnn_input, hidden)

        # Final output for current timestep
        output = self.fc_out(torch.cat((output.squeeze(1), context.squeeze(1)), dim=1))  # (batch_size, output_vocab_size)

        if self.cell_type == 'lstm':
            return output, (hidden, cell)
        else:
            return output, hidden


# Complete Seq2Seq model with attention
class Seq2Seq(nn.Module):
    def __init__(self, input_vocab_size, output_vocab_size, hidden_size, embedding_size,
                 encoder_layers=1, decoder_layers=1, dropout=0.3, cell_type='gru', bidirectional=True):
        super(Seq2Seq, self).__init__()

        # Initialize encoder and attention modules
        self.encoder = Encoder(input_vocab_size, hidden_size, embedding_size, encoder_layers, dropout, cell_type, bidirectional)
        attn_input_size = hidden_size * 2 if bidirectional else hidden_size
        self.attention = Attention(attn_input_size)

        # Initialize decoder
        self.decoder = DecoderWithAttention(attn_input_size, embedding_size, output_vocab_size,
                                            self.attention, decoder_layers, dropout, cell_type)

    def forward(self, source_seq, target_seq, teacher_forcing_ratio=0.5):
        # source_seq: (batch_size, source_seq_len)
        # target_seq: (batch_size, target_seq_len)
        batch_size = source_seq.size(0)
        target_len = target_seq.size(1)
        vocab_size = self.decoder.embedding.num_embeddings

        # Placeholder for decoder outputs
        outputs = torch.zeros(batch_size, target_len, vocab_size).to(source_seq.device)

        # Encoder forward pass
        encoder_outputs, encoder_hidden = self.encoder(source_seq)
        decoder_hidden = self._init_decoder_hidden(encoder_hidden)
        decoder_input = target_seq[:, 0]  # start token for decoder

        # Step through the decoder one token at a time
        for t in range(1, target_len):
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
            outputs[:, t] = decoder_output

            # Decide whether to use teacher forcing
            teacher_force = torch.rand(1).item() < teacher_forcing_ratio
            top1 = decoder_output.argmax(1)
            decoder_input = target_seq[:, t] if teacher_force else top1

        return outputs

    def _init_decoder_hidden(self, encoder_hidden):
        # Adjust encoder final hidden state to be used as decoder initial state
        if self.encoder.cell_type == 'lstm':
            h, c = encoder_hidden
            h = torch.cat([h[i] for i in range(h.shape[0])], dim=1).unsqueeze(0)
            c = torch.cat([c[i] for i in range(c.shape[0])], dim=1).unsqueeze(0)
            return (h, c) if h.shape[0] == self.decoder.num_layers else (h[:self.decoder.num_layers], c[:self.decoder.num_layers])
        else:
            hidden = torch.cat([encoder_hidden[i] for i in range(encoder_hidden.shape[0])], dim=1).unsqueeze(0)
            return hidden if hidden.shape[0] == self.decoder.num_layers else hidden[:self.decoder.num_layers]


In [33]:
def train(model, dataloader, criterion, optimizer, device):
    model.train()  # Set model to training mode
    total_loss = 0
    total_correct = 0
    total_samples = 0

    for source_seq, target_seq in tqdm(dataloader, desc='Training', unit='batch'):
        # Move tensors to the selected device (CPU/GPU)
        source_seq = source_seq.to(device)
        target_seq = target_seq.to(device)

        optimizer.zero_grad()  # Clear previous gradients

        # Forward pass with teacher forcing
        output = model(source_seq, target_seq)  # output: (batch, target_len, vocab_size)

        output_dim = output.shape[-1]

        # Flatten outputs and targets for loss computation
        output_flat = output.view(-1, output_dim)           # (batch * target_len, vocab_size)
        target_flat = target_seq.view(-1)                   # (batch * target_len)

        loss = criterion(output_flat, target_flat)          # Cross-entropy loss
        total_loss += loss.item()

        loss.backward()              # Backward pass
        optimizer.step()             # Update weights

        # Calculate word-level accuracy
        output_indices = output.argmax(dim=2)               # (batch, target_len)
        correct = (output_indices == target_seq).all(dim=0) # True if all tokens match for the sequence
        total_correct += correct.sum().item()
        total_samples += target_seq.size(0)

    accuracy = 100*(total_correct / total_samples)
    return model, total_loss / len(dataloader), accuracy


def evaluate(model, dataloader, criterion, device):
    model.eval()  # Set model to evaluation mode
    total_loss = 0
    total_correct = 0
    total_samples = 0

    with torch.no_grad():
        for source_seq, target_seq in tqdm(dataloader, desc='Evaluating', unit='batch'):
            source_seq = source_seq.to(device)
            target_seq = target_seq.to(device)

            # Forward pass without teacher forcing
            output = model(source_seq, target_seq, teacher_forcing_ratio=0.0)

            output_dim = output.shape[-1]
            output_flat = output.view(-1, output_dim)
            target_flat = target_seq.view(-1)

            loss = criterion(output_flat, target_flat)
            total_loss += loss.item()

            # Get predicted token indices
            output_indices = output.argmax(dim=2)  # (batch, target_len)

            # Optional fix: adjust prediction index values if needed
            # (this appears to be a task-specific fix for index shift)
            mask = output_indices > 9
            output_indices[mask] -= 2

            # Word-level accuracy: full sequence must match
            correct = (output_indices == target_seq).all(dim=0)
            total_correct += correct.sum().item()
            total_samples += target_seq.size(0)

    avg_loss = total_loss / len(dataloader)
    accuracy = 100*(total_correct / total_samples)
    return avg_loss, accuracy


In [11]:
# === Model Configuration ===
source_vocab_size = 30        # Number of unique characters in the Latin script (input)
target_vocab_size = 60        # Number of unique characters in the Devanagari script (output)

embedding_size = 16           # Size of the embedding vectors for both encoder and decoder
hidden_size = 32              # Number of hidden units in the RNN cells

encoder_layers = 1            # Number of RNN layers in the encoder
decoder_layers = 1            # Number of RNN layers in the decoder

rnn_cell_type = 'rnn'         # RNN cell type: 'rnn', 'gru', or 'lstm'
dropout1 = 0.3                 # Dropout probability to prevent overfitting

batch_size = 64               # Number of samples per batch
num_epochs = 12               # Total number of training epochs
learning_rate = 0.001         # Learning rate for the optimizer

# === Model Initialization ===
# Instantiate the Seq2Seq model with attention
model = Seq2Seq(
    input_vocab_size=source_vocab_size,
    output_vocab_size=target_vocab_size,
    hidden_size=hidden_size,
    embedding_size=embedding_size,
    encoder_layers=encoder_layers,
    decoder_layers=decoder_layers,
    dropout=dropout1,
    cell_type=rnn_cell_type,
    bidirectional=True  # Can be toggled off if bidirectional encoding is not desired
)

# Display model architecture
print(model)


Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(30, 16)
    (dropout): Dropout(p=0.3, inplace=False)
    (rnn): RNN(16, 32, batch_first=True, dropout=0.3, bidirectional=True)
  )
  (attention): Attention(
    (attn_layer): Linear(in_features=128, out_features=64, bias=True)
  )
  (decoder): DecoderWithAttention(
    (attention): Attention(
      (attn_layer): Linear(in_features=128, out_features=64, bias=True)
    )
    (embedding): Embedding(60, 16)
    (dropout): Dropout(p=0.3, inplace=False)
    (rnn): RNN(80, 64, batch_first=True, dropout=0.3)
    (fc_out): Linear(in_features=128, out_features=60, bias=True)
  )
)


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn(


In [12]:
wandb.login(key='53b259076c07d0811d73bf26bfef7437e04dbf66')

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ma23m013 (ma23m013-iit-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [13]:
# Sweep configuration for hyperparameter optimization using Weights & Biases
sweep_config = {
    'method': 'bayes',  # Bayesian optimization for more efficient search
    'metric': {
        'name': 'val_accuracy',  # Metric to optimize
        'goal': 'maximize'       # Maximize validation accuracy
    },
    'parameters': {
        'embedding_size': {
            'values': [16,32, 64, 128, 256]  # Size of embedding vectors
        },
        'dropout': {
            'values': [0.0,0.2,0.4]  # Dropout probability to reduce overfitting
        },
        'encoder_layers': {
            'values': [1]  # Number of RNN layers in the encoder
        },
        'decoder_layers': {
            'values': [1]  # Number of RNN layers in the decoder
        },
        'hidden_size': {
            'values': [16,32, 64, 128, 256]  # Hidden state dimension in RNN
        },
        'rnn_cell_type': {
            'values': ['lstm','gru','rnn']  # RNN cell type to use
        },
        'use_bidirectional_encoder': {
            'values': [True, False]  # Whether encoder is bidirectional
        },
        'batch_size': {
            'values': [32, 64]  # Batch size during training
        },
        'num_epochs': {
            'values': [8,10]  # Number of training epochs
        },
        'learning_rate': {
            'values': [0.01, 0.001]  # Learning rate for optimizer
        }
    }
}

# Launch sweep on Weights & Biases
sweep_id = wandb.sweep(sweep=sweep_config, project='DA6401_A-3_Attention')


Create sweep with ID: zzsnb7wu
Sweep URL: https://wandb.ai/ma23m013-iit-madras/DA6401_A-3_Attention/sweeps/zzsnb7wu


In [14]:
def run_training():
    '''
    This function is executed by WandB for each set of hyperparameters during the sweep.
    It initializes the model using the current configuration, trains and evaluates it,
    and logs the metrics to Weights & Biases.
    '''
    with wandb.init() as run:
        # Create a descriptive run name using the current hyperparameters
        run_name = (
            f"cell-{wandb.config.rnn_cell_type}"
            f"_encLayers-{wandb.config.encoder_layers}"
            f"_decLayers-{wandb.config.decoder_layers}"
            f"_dropout-{wandb.config.dropout}"
            f"_embedSize-{wandb.config.embedding_size}"
            f"_hiddenSize-{wandb.config.hidden_size}"
            f"_batchSize-{wandb.config.batch_size}"
            f"_epochs-{wandb.config.num_epochs}"
            f"_lr-{wandb.config.learning_rate}"
        )
        wandb.run.name = run_name

        # Instantiate model
        model = Seq2Seq(
            input_vocab_size=30,
            output_vocab_size=60,
            hidden_size=wandb.config.hidden_size,
            embedding_size=wandb.config.embedding_size,
            encoder_layers=wandb.config.encoder_layers,
            decoder_layers=wandb.config.decoder_layers,
            dropout=wandb.config.dropout,
            cell_type=wandb.config.rnn_cell_type,
            bidirectional=wandb.config.use_bidirectional_encoder
        )
        print(model)

        # Setup
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=wandb.config.learning_rate)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model.to(device)

        # Load data
        _, train_loader, _, _, _, _ = load_data(train_input,train_output, batch_size=wandb.config.batch_size)

        _, val_loader, _, _, _, _ = load_data(val_input,val_output, batch_size=wandb.config.batch_size)

        # Train & evaluate
        for epoch in range(wandb.config.num_epochs):
            model, train_loss, train_accuracy = train(model, train_loader, criterion, optimizer, device)
            val_loss, val_accuracy = evaluate(model, val_loader, criterion, device)

            # Log metrics
            wandb.log({
                'Epoch': epoch,
                'Train Loss': train_loss,
                'Train Accuracy (%)': train_accuracy,
                'Validation Loss': val_loss,
                'Validation Accuracy (%)': val_accuracy
            })

            print(f"Epoch {epoch + 1}/{wandb.config.num_epochs} | "
                  f"Train Loss: {train_loss:.4f} | Train Accuracy: {train_accuracy:.2f}% | "
                  f"Val Loss: {val_loss:.4f} | Val Accuracy: {val_accuracy:.2f}%")

# Start the sweep
wandb.agent(sweep_id, function=run_training, count=1)
wandb.finish()


wandb: Agent Starting Run: 1m2n448v with config:
wandb: 	batch_size: 64
wandb: 	decoder_layers: 1
wandb: 	dropout: 0
wandb: 	embedding_size: 128
wandb: 	encoder_layers: 1
wandb: 	hidden_size: 256
wandb: 	learning_rate: 0.01
wandb: 	num_epochs: 8
wandb: 	rnn_cell_type: lstm
wandb: 	use_bidirectional_encoder: True


Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(30, 128)
    (dropout): Dropout(p=0, inplace=False)
    (rnn): LSTM(128, 256, batch_first=True, bidirectional=True)
  )
  (attention): Attention(
    (attn_layer): Linear(in_features=1024, out_features=512, bias=True)
  )
  (decoder): DecoderWithAttention(
    (attention): Attention(
      (attn_layer): Linear(in_features=1024, out_features=512, bias=True)
    )
    (embedding): Embedding(60, 128)
    (dropout): Dropout(p=0, inplace=False)
    (rnn): LSTM(640, 512, batch_first=True)
    (fc_out): Linear(in_features=1024, out_features=60, bias=True)
  )
)


Evaluating: 100%|██████████| 163/163 [00:02<00:00, 76.45batch/s]


Epoch 1/8 | Train Loss: 1.0979 | Train Accuracy: 12.17% | Val Loss: 2.0679 | Val Accuracy: 8.18%


Evaluating: 100%|██████████| 163/163 [00:02<00:00, 78.59batch/s]


...

Evaluating: 100%|██████████| 163/163 [00:02<00:00, 78.11batch/s]


Epoch 7/8 | Train Loss: 0.9504 | Train Accuracy: 11.92% | Val Loss: 2.0978 | Val Accuracy: 5.79%


Evaluating: 100%|██████████| 163/163 [00:02<00:00, 74.03batch/s]

Epoch 8/8 | Train Loss: 0.9303 | Train Accuracy: 12.00% | Val Loss: 2.0888 | Val Accuracy: 5.31%


Epoch,▁▂▃▄▅▆▇█
Train Accuracy (%),█▁▂▄▆▃▁▃
Train Loss,█▅▅▄▃▃▂▁
Validation Accuracy (%),█▅▇█▇▁▂▁
Validation Loss,▁▂▅█▃▃▂▂
Epoch,7
Train Accuracy (%),12.0038
Train Loss,0.93034
Validation Accuracy (%),5.30507
Validation Loss,2.08876


# Test Accuracy for Best Model configuration

In [13]:
# Best Model configuration
input_vocab_size = 30         # Number of Latin script characters
output_vocab_size = 58        # Number of Urdu script characters
embedding_dim = 64           # Embedding dimension for both encoder and decoder
hidden_size = 128             # Hidden state size for RNN cells
encoder_layers = 1            # Number of layers in encoder RNN
decoder_layers = 1            # Number of layers in decoder RNN
cell_type = 'lstm'            # RNN cell type: 'rnn', 'gru', or 'lstm'
batch_size = 64               # Batch size during training
num_epochs = 10               # Total number of training epochs
dropout = 0.3                 # Dropout probability
learning_rate = 0.001         # Learning rate for optimizer
bidirectional = True          # Whether the encoder is bidirectional

# Initialize model, loss function, and optimizer
model = Seq2Seq(input_vocab_size, output_vocab_size, hidden_size, embedding_dim,
                encoder_layers, decoder_layers, dropout, cell_type, bidirectional)

print(model)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn(


Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(30, 64)
    (dropout): Dropout(p=0.3, inplace=False)
    (rnn): LSTM(64, 128, batch_first=True, dropout=0.3, bidirectional=True)
  )
  (attention): Attention(
    (attn_layer): Linear(in_features=512, out_features=256, bias=True)
  )
  (decoder): DecoderWithAttention(
    (attention): Attention(
      (attn_layer): Linear(in_features=512, out_features=256, bias=True)
    )
    (embedding): Embedding(58, 64)
    (dropout): Dropout(p=0.3, inplace=False)
    (rnn): LSTM(320, 256, batch_first=True, dropout=0.3)
    (fc_out): Linear(in_features=512, out_features=58, bias=True)
  )
)


In [14]:
# Load data
_, train_loader, _, _, _, _ = load_data(train_input,train_output, batch_size=64)

# Train
for epoch in range(num_epochs):
    model, train_loss, train_accuracy = train(model, train_loader, criterion, optimizer, device)

    print(f"Epoch {epoch + 1}/{num_epochs} | Train Loss: {train_loss:.4f} | Train Accuracy: {train_accuracy:.2f}% | ")
    

Training: 100%|██████████| 1661/1661 [01:03<00:00, 26.30batch/s]


Epoch 1/10 | Train Loss: 1.0818 | Train Accuracy: 36.94% | 


Training: 100%|██████████| 1661/1661 [01:02<00:00, 26.55batch/s]


Epoch 2/10 | Train Loss: 0.7826 | Train Accuracy: 37.40% | 


Training: 100%|██████████| 1661/1661 [01:02<00:00, 26.46batch/s]


...

Epoch 8/10 | Train Loss: 0.6223 | Train Accuracy: 37.97% | 


Training: 100%|██████████| 1661/1661 [01:04<00:00, 25.84batch/s]


Epoch 9/10 | Train Loss: 0.6083 | Train Accuracy: 38.03% | 


Training: 100%|██████████| 1661/1661 [01:03<00:00, 26.23batch/s]

Epoch 10/10 | Train Loss: 0.5982 | Train Accuracy: 38.04% | 


In [15]:
# Load data
_, test_loader, _, _, _, _ = load_data(test_input,test_output, batch_size=64)
test_loss, test_accuracy = evaluate(model, test_loader, criterion, device)
print(f"Test Accuracy: {test_accuracy:.2f}% | ")
 

Evaluating: 100%|██████████| 165/165 [00:01<00:00, 87.12batch/s]

Test Accuracy: 30.43% | 


In [16]:
def decode_prediction_indices(predicted_indices, index_to_token, target_vocab):
    """
    Converts a list of predicted token indices into a decoded string,
    excluding special tokens like <pad>, <sos>, and <eos>.
    """
    filtered_indices = []
    for idx in predicted_indices:
        if idx in index_to_token and idx not in (target_vocab['<pad>'], target_vocab['<sos>'], target_vocab['<eos>']):
            filtered_indices.append(idx)
    
    decoded_string = ''
    for idx in filtered_indices:
        decoded_string += index_to_token[idx]
    
    return decoded_string


def decode_target_indices(target_indices, index_to_token, target_vocab):
    """
    Converts target indices to a decoded string, adjusting indices if needed.
    Handles specific logic to shift indices >= 10 by -3.
    """
    filtered_indices = []
    for idx in target_indices:
        if idx in index_to_token and idx not in (target_vocab['<pad>'], target_vocab['<sos>'], target_vocab['<eos>']):
            corrected_idx = idx if idx < 10 else idx - 3
            filtered_indices.append(corrected_idx)
    
    decoded_string = ''
    for idx in filtered_indices:
        decoded_string += index_to_token[idx]
    
    return decoded_string


def predict_sequences(model, dataloader, device):
    """
    Uses the trained model to predict output sequences from input batches.
    Returns:
    - A list of tuples containing (input_sequence, predicted_output)
    - A list of ground truth target sequences
    """
    model.eval()
    predictions = []
    ground_truths = []

    with torch.no_grad():
        for source_batch, target_batch in dataloader:
            source_batch = source_batch.to(device)
            target_batch = target_batch.to(device)

            # Get model output with no teacher forcing
            output_logits = model(source_batch, target_batch, teacher_forcing_ratio=0.0)
            predicted_tokens = output_logits.argmax(dim=2)  # (batch_size, target_len)

            # Move tensors to CPU for further processing
            predictions.append((source_batch.cpu().numpy(), predicted_tokens.cpu().numpy()))
            ground_truths.append(target_batch.cpu().numpy())

    return predictions, ground_truths


In [17]:
dataset, dataloader, input_vocab, target_vocab, max_input_len, max_target_len = load_data(test_input, test_output,batch_size = 64)
# Make sure to define the reverse dictionaries for converting indices back to text
latin_idx2token = {idx: char for char, idx in input_vocab.items()}
urdu_idx2token = {idx: char for char, idx in target_vocab.items()}

In [18]:
# Run predictions on the Urdu test set using the trained model
predicted_batches, ground_truth_batches = predict_sequences(model, test_loader, device)

# This will store final results for each sequence: [input_text, ground_truth_text, predicted_text]
sequence_translation_results = []

# Iterate over batches of predictions and corresponding ground truth
for (batch_input_indices, batch_predicted_indices), batch_actual_indices in zip(predicted_batches, ground_truth_batches):

    # Iterate through each sample in the batch
    for sample_idx in range(batch_input_indices.shape[0]):
        # Decode the Latin-script input indices into a readable string
        input_text = decode_prediction_indices(
            batch_input_indices[sample_idx],
            latin_idx2token,
            input_vocab
        )

        # Decode the actual Urdu target indices into a readable string
        actual_text = decode_prediction_indices(
            batch_actual_indices[sample_idx],
            urdu_idx2token,
            target_vocab
        )

        # Decode the predicted Urdu output indices into a readable string (handling index shift if needed)
        predicted_text = decode_target_indices(
            batch_predicted_indices[sample_idx],
            urdu_idx2token,
            target_vocab
        )

        # Save the result for later inspection or logging
        sequence_translation_results.append([input_text, actual_text, predicted_text])


In [19]:
with open('predictions_attention.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Input Text', 'Actual Target', 'Predicted Text'])
    writer.writerows(sequence_translation_results)

In [20]:
df1 = pd.read_csv('predictions_attention.csv')
df1

,Input Text,Actual Target,Predicted Text
0,aaieen,آئین,اہك
1,aaoin,آئین,امئہ
2,aayean,آئین,آہاک
3,aayeen,آئین,آئہک
4,aayin,آئین,ائہک
...,...,...,...
10512,yegangat,یگانگت,ہڑكڑ
10513,ehan,یہان,ہںاک
10514,yahaan,یہان,ہںاک
10515,yahan,یہان,ہںاک


In [29]:

def visualize_attention(model, test_loader, device, input_vocab, output_vocab, num_examples=9):
    model.eval()
    fig, axs = plt.subplots(3, 3, figsize=(15, 12))
    axs = axs.flatten()
    
    special_tokens = {"<pad>", "<sos>", "<eos>"}
    count = 0

    with torch.no_grad():
        for source_seq, target_seq in test_loader:
            source_seq = source_seq.to(device)
            target_seq = target_seq.to(device)

            batch_size = source_seq.size(0)

            for i in range(batch_size):
                if count >= num_examples:
                    break

                src = source_seq[i].unsqueeze(0)
                tgt = target_seq[i].unsqueeze(0)

                encoder_outputs, encoder_hidden = model.encoder(src)
                decoder_hidden = model._init_decoder_hidden(encoder_hidden)
                decoder_input = tgt[:, 0]  # <sos>

                attn_weights_all = []

                for t in range(1, tgt.size(1)):
                    decoder_output, decoder_hidden = model.decoder(decoder_input, decoder_hidden, encoder_outputs)

                    if model.encoder.cell_type == 'lstm':
                        hidden_state = decoder_hidden[0][-1]
                    else:
                        hidden_state = decoder_hidden[-1]

                    attn_weights = model.attention(hidden_state, encoder_outputs)
                    attn_weights_all.append(attn_weights.cpu().squeeze().numpy())

                    decoder_input = tgt[:, t]

                attn_matrix = np.stack(attn_weights_all, axis=0)

                src_token_ids = src.squeeze().tolist()
                tgt_token_ids = tgt.squeeze()[1:].tolist()

                src_tokens = [input_vocab.get(tok, "<unk>") for tok in src_token_ids]
                tgt_tokens = [output_vocab.get(tok, "<unk>") for tok in tgt_token_ids]

                filtered_src = [(idx, tok) for idx, tok in enumerate(src_tokens) if tok not in special_tokens]
                filtered_tgt = [(idx, tok) for idx, tok in enumerate(tgt_tokens) if tok not in special_tokens]

                if not filtered_src or not filtered_tgt:
                    continue

                src_indices, src_labels = zip(*filtered_src)
                tgt_indices, tgt_labels = zip(*filtered_tgt)

                attn_matrix_filtered = attn_matrix[np.array(tgt_indices)[:, None], np.array(src_indices)]

                ax = axs[count]
                im = ax.imshow(attn_matrix_filtered, cmap="viridis")
                ax.set_title(f"Sample {count+1}")
                ax.set_xlabel("Input Tokens")
                ax.set_ylabel("Output Tokens")
                ax.set_xticks(np.arange(len(src_labels)))
                ax.set_yticks(np.arange(len(tgt_labels)))
                ax.set_xticklabels(src_labels, rotation=90)
                ax.set_yticklabels(tgt_labels)

                count += 1

            if count >= num_examples:
                break

    plt.tight_layout()

    # Log the combined image to wandb
    wandb.log({"Combined Attention Heatmaps": wandb.Image(fig)})
    plt.close(fig)


In [32]:
wandb.init(project="DA6401_A-3_Attention", name="attention-visualization-run")
visualize_attention(model, test_loader, device, latin_idx2token,urdu_idx2token, num_examples=9)
wandb.finish()
